<a href="https://colab.research.google.com/github/Tung0000/Opencv2_TestVideo_Colab/blob/main/Opencv2_test_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Test Video

In [ ]:
soure="/content/2583475-hd_1920_1080_24fps.mp4"
cap=cv2.VideoCapture(soure)

if not cap.isOpened():
  raise RuntimeError("วิดิโอเล่นไม่ได้")

while True:
  check,frame =cap.read()
  if not check:break
  hsv_img=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
  gray=cv2.cvtColor(hsv_img,cv2.COLOR_BGR2GRAY)
  if cv2.waitKey(1) & 0xFF == ord('q'):
    break
  cv2_imshow(gray)

cap.release()
cv2.destroyAllWindows()

#ใช้ hsv ตรวจจับสีที่ต้องการในวิดิโอ

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow # สำหรับใช้ใน Google Colab

soure = "/content/2583475-hd_1920_1080_24fps.mp4"
cap = cv2.VideoCapture(soure)

if not cap.isOpened():
    raise RuntimeError("วิดีโอเล่นไม่ได้ หรือพาธไฟล์ไม่ถูกต้อง")

# กำหนดช่วงสีแดง (HSV)
lower_red_1 = np.array([0, 100, 100])
upper_red_1 = np.array([10, 255, 255])
lower_red_2 = np.array([160, 100, 100])
upper_red_2 = np.array([179, 255, 255])

print("กำลังประมวลผลวิดีโอ... โปรดรอภาพแสดง")

while True:
    check, frame = cap.read()
    if not check:
        break

    # 1. การประมวลผลสีแดง
    hsv_img = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask1 = cv2.inRange(hsv_img, lower_red_1, upper_red_1)
    mask2 = cv2.inRange(hsv_img, lower_red_2, upper_red_2)
    mask = cv2.bitwise_or(mask1, mask2)#เอา mask1 mask2มารวมกัน
    resuls = cv2.bitwise_and(frame, frame, mask=mask)

    # 2. แก้ไขปัญหา Channels และการรวมภาพ
    # ต้องแปลง Mask (1 Channel) ให้มี 3 Channels ก่อนจึงจะรวมกับ Frame (3 Channels) ได้
    mask_3_channel = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

    # รวม Original Frame, Mask 3 Channels, และ Result เข้าด้วยกันในแนวนอน
    # เราตั้งชื่อตัวแปรผลลัพธ์ว่า 'output_stack'
    output_stack = np.hstack([frame, mask_3_channel, resuls])

    # 3. แสดงผลลัพธ์
    cv2_imshow(output_stack)

    # หน่วงเวลาและรอการกด 'q' (ใน Colab ควรใช้ค่าหน่วงเวลาน้อยๆ)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 4. ปิดการใช้งาน (ต้องใช้ () หลัง release)
cap.release()
cv2.destroyAllWindows()

#ใช้ erode

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow # สำหรับใช้ใน Google Colab

soure = "/content/2583475-hd_1920_1080_24fps.mp4"
cap = cv2.VideoCapture(soure)

if not cap.isOpened():
    raise RuntimeError("วิดีโอเล่นไม่ได้ หรือพาธไฟล์ไม่ถูกต้อง")

# 1. กำหนดค่าสีแดง (HSV) และ Kernel
lower_red_1 = np.array([0, 100, 100])
upper_red_1 = np.array([10, 255, 255])
lower_red_2 = np.array([160, 100, 100])
upper_red_2 = np.array([179, 255, 255])

# Kernel สำหรับ Morphological Operation (เช่น Erode, Dilate)
# สำหรับ Erosion ควรใช้ Kernel ที่มีค่าเป็น 1 ทั้งหมด (ไม่ใช่ค่าเฉลี่ย 1/100)
kernal = np.ones((5, 5), np.uint8) # ใช้ขนาด 5x5 และค่าเป็น 1 (Uint8)

print("กำลังประมวลผลวิดีโอ... (เพิ่ม Erosion)")

while True:
    check, frame = cap.read()
    if not check:
        break

    # 2. การสร้าง Mask สีแดง
    hsv_img = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask1 = cv2.inRange(hsv_img, lower_red_1, upper_red_1)
    mask2 = cv2.inRange(hsv_img, lower_red_2, upper_red_2)
    mask = cv2.bitwise_or(mask1, mask2)

    # 3. >>> การปรับปรุง Mask ด้วย Erosion (ส่วนที่คุณต้องการเพิ่ม) <<<
    # Erode จะ 'กัดเซาะ' ขอบของวัตถุและลบ Noise เล็ก ๆ ออกไป
    eroded_mask = cv2.erode(mask, kernal, iterations=2)

    # 4. สร้างภาพผลลัพธ์ (Result) โดยใช้ Mask ที่ถูกปรับปรุงแล้ว
    resuls = cv2.bitwise_and(frame, frame, mask=eroded_mask)

    # 5. การรวมภาพเพื่อแสดงผล
    # เราใช้ eroded_mask (ภาพที่ผ่านการปรับปรุง) ในการแสดงผลแทน mask เดิม
    mask_3_channel = cv2.cvtColor(eroded_mask, cv2.COLOR_GRAY2BGR)
    stack = np.hstack([frame, mask_3_channel, resuls])

    cv2_imshow(stack)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# ปิดการใช้งาน
cap.release()
cv2.destroyAllWindows()

#ใช้Open Close

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

soure = "/content/2583475-hd_1920_1080_24fps.mp4"
cap = cv2.VideoCapture(soure)

if not cap.isOpened():
    raise RuntimeError("วิดีโอเล่นไม่ได้ หรือพาธไฟล์ไม่ถูกต้อง")

lower_red_1 = np.array([0, 100, 100])
upper_red_1 = np.array([10, 255, 255])
lower_red_2 = np.array([160, 100, 100])
upper_red_2 = np.array([179, 255, 255])


kernal = np.ones((5, 5), np.uint8)

print("กำลังประมวลผลวิดีโอ... (เพิ่ม Erosion)")

while True:
    check, frame = cap.read()
    if not check:
        break

    # 2. การสร้าง Mask สีแดง
    hsv_img = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask1 = cv2.inRange(hsv_img, lower_red_1, upper_red_1)
    mask2 = cv2.inRange(hsv_img, lower_red_2, upper_red_2)
    mask = cv2.bitwise_or(mask1, mask2)

    # 3. >>> การปรับปรุง Mask
    # สร้าง Open โดยใช้ mask
    open_close = cv2.morphologyEx(mask, cv2.MORPH_OPEN,kernal, iterations=2)
    open_close = cv2.morphologyEx(mask, cv2.MORPH_CLOSE,kernal, iterations=2)
    # 4. สร้าง Close โดยใช้ mask
    resuls = cv2.bitwise_and(frame, frame, mask=mask)

    # 5. การรวมภาพเพื่อแสดงผล
    # เราใช้ eroded_mask (ภาพที่ผ่านการปรับปรุง) ในการแสดงผลแทน mask เดิม
    mask_3_channel = cv2.cvtColor(open_close, cv2.COLOR_GRAY2BGR)
    stack = np.hstack([frame, mask_3_channel, resuls])

    cv2_imshow(stack)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# ปิดการใช้งาน
cap.release()
cv2.destroyAllWindows()

#ใช้ Threshold

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

soure = "/content/2583475-hd_1920_1080_24fps.mp4"
cap = cv2.VideoCapture(soure)

if not cap.isOpened():
    raise RuntimeError("วิดีโอเล่นไม่ได้ หรือพาธไฟล์ไม่ถูกต้อง")

lower_red_1 = np.array([0, 100, 100])
upper_red_1 = np.array([10, 255, 255])
lower_red_2 = np.array([160, 100, 100])
upper_red_2 = np.array([179, 255, 255])

kernal = np.ones((5, 5), np.uint8)

print("กำลังประมวลผลวิดีโอ...")

while True:
    check, frame = cap.read()
    if not check:
        break

    # 1. สร้าง Mask สีแดง (Mask ดิบ)
    hsv_img = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask1 = cv2.inRange(hsv_img, lower_red_1, upper_red_1)
    mask2 = cv2.inRange(hsv_img, lower_red_2, upper_red_2)
    mask = cv2.bitwise_or(mask1, mask2)

    # --- 2. ขั้นตอนการปรับปรุง Mask (Refinement) ---

    # 2.1 Opening (เปิด): ลบ Noise เล็กๆ
    refined_mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernal, iterations=2)

    # 2.2 Closing (ปิด): เติมรูโหว่เล็กๆ (ใช้ผลลัพธ์จาก Opening ต่อไป)
    refined_mask = cv2.morphologyEx(refined_mask, cv2.MORPH_CLOSE, kernal, iterations=1)

    # 2.3 Threshold (ทางเลือก): หากต้องการบังคับให้ค่าเป็น 0 หรือ 255 แน่นอน
    # Note: เนื่องจาก Mask มาจาก cv2.inRange() ซึ่งเป็น Binary อยู่แล้ว
    # เราจึงใช้ THRESH_BINARY ธรรมดา (Threshold 127) หรือจะข้ามขั้นตอนนี้ก็ได้
    _, refined_mask = cv2.threshold(refined_mask, 127, 255, cv2.THRESH_BINARY)

    # 3. สร้างภาพผลลัพธ์ (Result)
    # ใช้ refined_mask ที่ผ่านการปรับปรุงแล้ว
    resuls = cv2.bitwise_and(frame, frame, mask=refined_mask)

    # 4. การเตรียมภาพสำหรับการรวม (hstack)
    # แปลง refined_mask (1 Channel) เป็น 3 Channels เพื่อแสดงผล
    mask_3_channel = cv2.cvtColor(refined_mask, cv2.COLOR_GRAY2BGR)

    # 5. รวมภาพเพื่อแสดงผล
    # แสดง: Original Frame | Refined Mask | Red Only Result
    stack = np.hstack([frame, mask_3_channel, resuls])

    cv2_imshow(stack)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# ปิดการใช้งาน
cap.release()
cv2.destroyAllWindows()

#รวมทุกขั้นตอนไว้ที่นี้และใช้ contours ในการตรวจจับ

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

soure = "/content/2583475-hd_1920_1080_24fps.mp4"
cap = cv2.VideoCapture(soure)

if not cap.isOpened():
    raise RuntimeError("วิดีโอเล่นไม่ได้ หรือพาธไฟล์ไม่ถูกต้อง")

lower_red_1 = np.array([0, 100, 100])
upper_red_1 = np.array([10, 255, 255])
lower_red_2 = np.array([160, 100, 100])
upper_red_2 = np.array([179, 255, 255])

kernal = np.ones((5, 5), np.uint8)

print("กำลังประมวลผลวิดีโอ...")

while True:
    check, frame = cap.read()
    if not check:
        break

    # 1. สร้าง Mask สีแดง (Mask ดิบ)
    hsv_img = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask1 = cv2.inRange(hsv_img, lower_red_1, upper_red_1)
    mask2 = cv2.inRange(hsv_img, lower_red_2, upper_red_2)
    mask = cv2.bitwise_or(mask1, mask2)

    # 2. ขั้นตอนการปรับปรุง Mask (Refinement)
    refined_mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernal, iterations=2)
    refined_mask = cv2.morphologyEx(refined_mask, cv2.MORPH_CLOSE, kernal, iterations=1)

    # 3. Threshold (บังคับให้เป็น Binary ชัดเจน)
    _, refined_mask = cv2.threshold(refined_mask, 127, 255, cv2.THRESH_BINARY)

    # 4. สร้างภาพผลลัพธ์ (Result: สีแดงบนพื้นดำ)
    resuls = cv2.bitwise_and(frame, frame, mask=refined_mask)

    # 5. >>> การหา Contours และ Bounding Box <<<
    # (Fix 1: รับค่าคืนจาก findContours 2 ตัว)
    contours, _ = cv2.findContours(refined_mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Copy ภาพต้นฉบับเพื่อวาด Bounding Box
    vis = frame.copy()

    for c in contours:
        area = cv2.contourArea(c)
        # กรอง Contours ที่เล็กเกินไป (Noise)
        if area < 4000: # คุณกำหนด 4000
            continue

        # หา Bounding Box
        x, y, w, h = cv2.boundingRect(c)
        # วาดสี่เหลี่ยมบนภาพ vis
        # (Fix 2: เปลี่ยนความหนาเป็น 2 หรือ 3 เพื่อให้เห็นชัด)
        cv2.rectangle(vis, (x, y), (x + w, y + h), (0, 255, 255), 2) # สีเหลือง, หนา 2

    # 6. การเตรียมภาพสำหรับการรวม (hstack)
    mask_3_channel = cv2.cvtColor(refined_mask, cv2.COLOR_GRAY2BGR)

    # 7. รวมภาพเพื่อแสดงผล (Fix 3: นำภาพ vis ที่มี Bounding Box มารวมด้วย)
    # แสดง: Original Frame | Refined Mask | Red Only Result | Bounding Box Frame
    stack = np.hstack([f vis])

    cv2_imshow(stack)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# ปิดการใช้งาน
cap.release()
cv2.destroyAllWindows()